# Stage 2a: Generate Alpha Factors using Stock Universe

- Compute custom apha factors
- Compute univeral quant factors
- Compute date information
- Save Alpha Factors

In [1]:
from platform import python_version
import time
from datetime import datetime
import os
import pandas as pd
import numpy as np
import math
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (20, 8)

# Set the import path for the tools directiory
import sys
# insert at position 1 in the path, as 0 is the path of this file.
sys.path.insert(1, '../tools')
import importlib
import ameritrade_functions as amc
importlib.reload(amc)
import utils
importlib.reload(utils)

print(f'Python version: {python_version()}')
print(f'Pandas version: {pd.__version__}')

Python version: 3.8.10
Pandas version: 0.25.3


## Configure Ameritrade Information

Ameritrade credentials are stored in environment variables to keep from having unencrypted passwords stored on disk.

The module automatically masks the account numbers to protect the actual accounts. An Ameritrade user can have many investment accounts. We will be working with only one for this demonstration.

In [2]:
username = os.getenv('maiotradeuser')
password = os.getenv('maiotradepw')
client_id = os.getenv('maiotradeclientid')

masked_account_number = '#---9216'
account_portfolios_file_name = 'data/portfolio_data.csv'
portfolio_file_name = 'data/portfolio_' + masked_account_number[-4:] + '.csv'
price_histories_file_name = 'data/price_histories.csv'

# Price History data from Stage 1

In [3]:
price_histories = utils.read_price_histories(price_histories_file_name)
print(f'Date range for price histories: {price_histories.date.min().date()} to {price_histories.date.max().date()}')
close = utils.get_close_values(price_histories)
print(f'You have {len(close.columns)} stocks')
print(close.columns.to_list())
close.tail()

Date range for price histories: 2016-09-02 to 2021-09-02
You have 436 stocks
['A', 'AAL', 'AAP', 'ABBV', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AIZ', 'AJG', 'ALB', 'ALGN', 'ALK', 'ALL', 'ALLE', 'AMAT', 'AMCR', 'AMD', 'AMGN', 'AMP', 'AMT', 'ANET', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'APD', 'APH', 'APTV', 'ARE', 'ATO', 'ATVI', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXP', 'AZO', 'BA', 'BAC', 'BAX', 'BBY', 'BDX', 'BEN', 'BIIB', 'BIO', 'BK', 'BKNG', 'BKR', 'BLK', 'BMY', 'BR', 'BSX', 'BWA', 'BXP', 'C', 'CAG', 'CAH', 'CARR', 'CAT', 'CB', 'CBOE', 'CBRE', 'CCI', 'CCL', 'CDNS', 'CDW', 'CE', 'CERN', 'CF', 'CFG', 'CHD', 'CHTR', 'CI', 'CINF', 'CLX', 'CMA', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COG', 'COO', 'COP', 'COST', 'CPB', 'CRL', 'CRM', 'CSCO', 'CSX', 'CTAS', 'CTLT', 'CTSH', 'CTXS', 'CVS', 'CVX', 'CZR', 'D', 'DAL', 'DD', 'DE', 'DG', 'DHI', 'DHR', 'DIS', 'DISCA', 'DISCK', 'DISH', 'DLR', 'DOV', 'DOW', 'DPZ', 'DRE', 'DRI', 'DTE', 'DUK',

ticker,A,AAL,AAP,ABBV,ABT,ACN,ADBE,ADI,ADM,ADP,...,WYNN,XEL,XLNX,XOM,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2021-08-27 00:00:00+00:00,173.81,20.220,203.82,119.58,124.59,335.66,658.52,166.48,60.57,208.96,...,101.05,68.70,156.27,55.770,134.94,131.70,150.07,587.96,58.55,203.82
2021-08-30 00:00:00+00:00,176.73,19.510,201.14,120.57,125.94,337.00,665.99,166.98,60.07,209.07,...,99.89,69.03,155.07,55.160,136.48,130.69,149.78,584.78,57.20,205.89
2021-08-31 00:00:00+00:00,175.47,19.940,202.85,120.78,126.37,336.56,663.70,162.95,60.00,209.04,...,101.69,68.75,155.59,54.520,136.31,131.03,150.45,587.17,57.90,204.56
2021-09-01 00:00:00+00:00,175.45,19.640,201.88,112.27,127.40,337.90,665.89,161.11,59.66,206.86,...,100.98,69.75,154.47,53.770,136.55,131.55,150.87,582.78,57.17,206.44
2021-09-02 00:00:00+00:00,177.66,19.755,201.14,111.36,127.95,340.31,663.54,162.19,59.93,206.27,...,102.10,70.07,153.60,54.985,137.30,131.16,148.99,587.92,57.07,208.55


# Factors

### General
- Momentum
- Mean Reversion
- Overnight Sentiment

### Universal Quant Features
- Volatility 
- Dollar Volume
- Market Dispersion
- Market Volatility
- Date Parts

#TODO: SMA for market disp/vol

In [5]:
import trading_factors as alpha_factors
importlib.reload(alpha_factors)

all_factors = pd.concat(
[
    alpha_factors.OvernightSentiment(price_histories, 5).demean().rank().zscore().smoothed(10).rank().zscore().for_al(),
    alpha_factors.FactorMeanReversion(price_histories, 5).demean().rank().zscore().for_al(),
    alpha_factors.AnnualizedVolatility(price_histories, 20).rank().zscore().for_al()
], axis=1)
all_factors.sort_index(inplace=True)
all_factors = all_factors.dropna()

all_factors.to_csv('data/all_factors.csv')

In [10]:
all_factors = pd.read_csv('data/all_factors.csv', parse_dates=['date'])
all_factors.set_index(['date', 'ticker'], inplace=True)
all_factors.sort_index(inplace=True)
all_factors.iloc[all_factors.index.get_level_values('ticker') == 'AAL'].tail()

,,overnight_sentiment_5_day_smoothed,mean_reversion_5_day_logret,annualzed_volatility_20_day
date,ticker,,,
2021-08-27 00:00:00+00:00,AAL,0.178767,-1.561233,1.513562
2021-08-30 00:00:00+00:00,AAL,0.250274,-0.639589,1.521507
2021-08-31 00:00:00+00:00,AAL,0.504521,0.131096,1.521507
2021-09-01 00:00:00+00:00,AAL,0.846165,1.283151,1.521507
2021-09-02 00:00:00+00:00,AAL,0.997124,1.100411,1.457946


In [7]:
all_factors.describe()

,overnight_sentiment_5_day_smoothed,mean_reversion_5_day_logret,annualzed_volatility_20_day
count,5.319200e+05,5.319200e+05,5.319200e+05
mean,1.302412e-19,-3.005567e-20,-2.905381e-19
std,1.000001e+00,1.000001e+00,1.000001e+00
min,-1.728083e+00,-1.728101e+00,-1.728083e+00
25%,-8.660278e-01,-8.700003e-01,-8.640433e-01
50%,0.000000e+00,0.000000e+00,0.000000e+00
75%,8.660277e-01,8.620619e-01,8.640433e-01
max,1.728083e+00,1.728101e+00,1.728088e+00


In [8]:
all_factors.head()

overnight_sentiment_5_day_smoothed  \
date                      ticker                                       
2016-10-28 00:00:00+00:00 A                                -0.528356   
                          AAL                              -1.124247   
                          AAP                              -1.465891   
                          ABBV                              0.131096   
                          ABT                              -1.402330   

                                  mean_reversion_5_day_logret  \
date                      ticker                                
2016-10-28 00:00:00+00:00 A                          1.386445   
                          AAL                       -0.520413   
                          AAP                        0.305892   
                          ABBV                       1.457952   
                          ABT                        0.806442   

                                  annualzed_volatility_20_day  
date                      ticker                               
2016-10-28 00:00:00+00:00 A                          0.981236  
                          AAL                        1.465895  
                          AAP                       -0.353563  
                          ABBV                      -0.576029  
                          ABT                        0.623701

In [9]:
all_factors.tail()

overnight_sentiment_5_day_smoothed  \
date                      ticker                                       
2021-09-02 00:00:00+00:00 YUM                              -1.354658   
                          ZBH                              -0.059589   
                          ZBRA                              1.044795   
                          ZION                             -0.258219   
                          ZTS                              -0.456849   

                                  mean_reversion_5_day_logret  \
date                      ticker                                
2021-09-02 00:00:00+00:00 YUM                        0.870000   
                          ZBH                        0.973288   
                          ZBRA                      -0.170822   
                          ZION                      -0.750822   
                          ZTS                       -0.615754   

                                  annualzed_volatility_20_day  
date                      ticker                               
2021-09-02 00:00:00+00:00 YUM                        1.553288  
                          ZBH                        0.560137  
                          ZBRA                      -0.313836  
                          ZION                       0.059589  
                          ZTS                       -1.473836